# Model Inputs

### Packages

In [1]:
# General
import pandas as pd
import numpy as np
import pandas as pd
import os
DATA_DIR = "~/Desktop/code/data/"

# Pipeline
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 

from sklearn.decomposition import PCA 
import umap
import hdbscan

from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

# Visualization
import plotly.express as px
import plotly.graph_objects as go

### Dataset

In [2]:
# Loading output dataframe from data.ipynb
df = pd.read_csv(os.path.join(DATA_DIR, "ml_input.csv"), low_memory=False)

# Filter non-specific ec numbers (e.g., 1.1.1,-)
filter = df["ec"].str.contains("-")
df = df[~filter]

# Format for machine learning
df = df[["taxon_id", "media_id", "ec"]].value_counts().reset_index()
df = df.pivot(index=["taxon_id", "media_id"], columns="ec", values="count")
df = df.fillna(0.0).reset_index()

# Filtering into training and test sets (database set vs. MAGs)
bins = df["media_id"].str.contains("unknown")
df_train = df[~bins]
df_test = df[bins]

# Model

### Training and test sets

In [ ]:
# Split the datasets into training and test sets
TARGET = "media_id" # target label
RDM = 787 # seed for random_state

# Test case with MAGs
X_train = df_train.drop(["taxon_id", "media_id"], axis=1)
y_train = df_train[TARGET]
X_test = df_test.drop(["taxon_id", "media_id"], axis=1)
y_test = df_test[TARGET]

### Pipeline

In [ ]:
# sklearn pipeline helps prevent data leakage; incorporate individual steps here
pipeline = Pipeline([
    ('scaler', StandardScaler()),    # Scale data (optional)
    ('pca', PCA(n_components=120)),   # Reduce dimensionality with PCA
    ('umap', umap.UMAP(              # Further reduce with UMAP
        metric="euclidean",
        n_components=2, #20 for clustering, 2 for visualization
        n_epochs=200, #200 recommended for large datasets, higher => stricter clustering
        random_state=RDM,
        n_jobs=1,
        n_neighbors=50,
        min_dist=0.2
        )) 
])

### Fit and transform

In [ ]:
# Fit and transform the pipeline on the training and test data
X_train_transformed = pipeline.fit_transform(X_train) # fit pipeline and transform training data
X_test_transformed = pipeline.transform(X_test) # transform test data

### Optimizing n_clusters

In [ ]:
# Silhouette coefficient method
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

# Test a range of clusters for their silhouette coefficients
clusters = []
for n_cluster in range(20, 100):
    kmeans = KMeans(n_clusters=n_cluster).fit(X_train_transformed)
    label = kmeans.labels_
    sil_coeff = silhouette_score(X_train_transformed, label, metric='euclidean')
    clusters.append({'n_clusters': n_cluster, 'coefficient': sil_coeff})

# Select the maximum coefficient
clusters = pd.DataFrame(clusters)
c = clusters.iloc[clusters["coefficient"].argmax()]["n_clusters"]
s = clusters["coefficient"].max()
n = int(c)

print("{} clusters returns a maximum Silhouette Coefficient of {}".format(n-1, s))

95 clusters returns a maximum Silhouette Coefficient of 0.6928007006645203


In [ ]:
# Cluster
clusterer = KMeans(n_clusters=n, random_state=RDM)
cluster_labels = clusterer.fit_predict(X_train_transformed)
test_clusters = clusterer.predict(X_test_transformed)

### Metrics

In [ ]:
# Initial dimensionality reduction performance (PCA)
pca = pipeline.named_steps['pca']
explained_variance = pca.explained_variance_ratio_
print(f"Total PCA explained variance: {explained_variance.sum()}")

# Further dimensionality reduction and clustering performance
silhouette_avg = silhouette_score(X_train_transformed, cluster_labels)
print(clusterer.labels_.max(), "KMeans training clusters")
print(f"Average Silhouette Score on KMeans clusters: {silhouette_avg}")
    # Silhouette score <=0.25 implies poor clustering, 0.25<x<=0.50 is fair, <50 is good

Total PCA explained variance: 0.8272737370027141
95 KMeans training clusters
Average Silhouette Score on KMeans clusters: 0.6907055974006653


In [ ]:
# Compare to hdbscan clustering 

# Apply HDBSCAN clustering on the transformed training data
hclusterer = hdbscan.HDBSCAN(
    min_samples=5, 
    min_cluster_size=15, 
    #gen_min_span_tree=True, 
    prediction_data=True # cluster_selection_method="leaf", for more fine-grained clustering
)
hcluster_labels = hclusterer.fit_predict(X_train_transformed)

# Clustering performance (UMAP and hdbscan)
hsilhouette_avg = silhouette_score(X_train_transformed, hcluster_labels)
print(hclusterer.labels_.max(), "hdbscan training clusters")
print(f"Silhouette Score on hdbscan clusters: {hsilhouette_avg}")

# Convert to numpy array for hdbscan and predict clustering
#test_points = test_df[["Component 1", "Component 2"]].to_numpy()
#test_labels, strengths = hdbscan.approximate_predict(clusterer, test_points)

# Optional constraints
#filtered = train_df[train_df['Cluster'] >= 0]
#test_labels = clusterer.fit_predict(X_test_transformed)

172 hdbscan training clusters
Silhouette Score on hdbscan clusters: 0.5683917999267578


### Visualization

In [ ]:
# Training data
train_df = pd.DataFrame(X_train_transformed, columns=['Component 1', 'Component 2'])
train_df['Set'] = 'Training'
train_df['Target'] = y_train
train_df['Cluster'] = cluster_labels # only showing training clusters for now, mapping test over these
train_df['taxon_id'] = df_train["taxon_id"]
train_df

,Component 1,Component 2,Set,Target,Cluster,taxon_id
0,8.765712,-20.685938,Training,J22,21,1002526
1,4.226567,-14.799174,Training,1a,6,1004166
2,31.787260,12.918801,Training,J181,11,1004261
3,31.795956,12.927306,Training,J455,11,1004261
4,8.807454,-20.657722,Training,J118,21,1005925
...,...,...,...,...,...,...
8427,20.777164,19.205803,Training,J12,78,"[{'NCBI tax id': 981331, 'Matching level': 'st..."
8428,7.116502,9.841452,Training,J12,90,"[{'NCBI tax id': 983594, 'Matching level': 'st..."
8429,7.082302,9.818217,Training,J22,90,"[{'NCBI tax id': 983594, 'Matching level': 'st..."
8430,7.115156,9.837695,Training,1a,90,"[{'NCBI tax id': 983920, 'Matching level': 'st..."


In [ ]:
# Media cluster labels
media_clusters = pd.read_csv(os.path.join(DATA_DIR, "comp-model.csv"))
media_clusters = media_clusters[["media_id", "Cluster"]]
media_clusters = media_clusters.rename(columns={"media_id": "Target", "Cluster": "mm_cluster"})
media_clusters

,Target,mm_cluster
0,J574,3
1,621a,4
2,J118,11
3,J42,1
4,J557,11
...,...,...
2601,J25,4
2602,J13,11
2603,J22,4
2604,35a,11


In [ ]:
target_label_mapping = dict(zip(media_clusters['Target'], media_clusters['mm_cluster']))
train_df['mm_cluster'] = train_df['Target'].map(target_label_mapping)
train_df

,Component 1,Component 2,Set,Target,Cluster,taxon_id,mm_cluster
0,8.765712,-20.685938,Training,J22,21,1002526,4.0
1,4.226567,-14.799174,Training,1a,6,1004166,11.0
2,31.787260,12.918801,Training,J181,11,1004261,4.0
3,31.795956,12.927306,Training,J455,11,1004261,2.0
4,8.807454,-20.657722,Training,J118,21,1005925,10.0
...,...,...,...,...,...,...,...
8427,20.777164,19.205803,Training,J12,78,"[{'NCBI tax id': 981331, 'Matching level': 'st...",11.0
8428,7.116502,9.841452,Training,J12,90,"[{'NCBI tax id': 983594, 'Matching level': 'st...",11.0
8429,7.082302,9.818217,Training,J22,90,"[{'NCBI tax id': 983594, 'Matching level': 'st...",4.0
8430,7.115156,9.837695,Training,1a,90,"[{'NCBI tax id': 983920, 'Matching level': 'st...",11.0


In [ ]:
# Test data
test_df = pd.DataFrame(X_test_transformed, columns=['Component 1', 'Component 2'])
test_df['Set'] = 'Test'
test_df['Target'] = list(df_test['media_id'])
test_df['Cluster'] = test_clusters
test_df['taxon_id'] = list(df_test['taxon_id'])
test_df.head()

,Component 1,Component 2,Set,Target,Cluster,taxon_id
0,8.017917,10.637187,Test,unknown,59,KBase_derived_Bin.001.fastaBA_F_extracted_bins...
1,8.316375,10.392190,Test,unknown,59,KBase_derived_Bin.002.fastaBA_F_extracted_bins...
2,7.567891,8.696896,Test,unknown,58,KBase_derived_Bin.004.fastaBA_F_extracted_bins...
3,8.184480,8.883637,Test,unknown,74,KBase_derived_Bin.005.fastaBA_F_extracted_bins...
4,7.587075,9.510571,Test,unknown,90,KBase_derived_Bin.006.fastaBA_F_extracted_bins...


In [ ]:
# Plot the training points
fig = px.scatter(
    data_frame = train_df,
    x = "Component 1",
    y = "Component 2",
    color = "Cluster",
    hover_data = ["taxon_id", "Target", "mm_cluster"],
    opacity = 0.3
) 

# Add the test points as a scatter trace
fig.add_trace(
    go.Scatter(
        x=test_df["Component 1"],  # x coordinates
        y=test_df["Component 2"],  # y coordinates
        mode='markers',
        marker=dict(
            color='black',  
            size=5,  
            opacity=1.0 
        ),
        text=test_df["Target"]  # marker hover text
    )
)

# Opacity slider
steps = []
for i in range(11):  # 0 to 1 in steps of 0.1
    step = dict(
        method="restyle",
        args=[{"marker.opacity": [i/20]}, [0]],  # Adjust opacity of the first trace only
        label=str(i/20)
    )
    steps.append(step)
sliders = [dict(
    active=3,  # initial value corresponding to opacity=0.3
    currentvalue={"prefix": "Opacity: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    title="KMeans",
    template="plotly_white",
)

fig.show()

# Classifier

### Nearest Neighbors

In [ ]:
#from sklearn.linear_model import LogisticRegression
#from scipy.stats import uniform

#from sklearn.model_selection import RandomizedSearchCV
#RandomizedSearchCV(KNearestNeighbors(),
#                                 param_grid={'knearestneighbors__n_neighbors': [5, 10, 40]},
#                                 cv=5,
#                                 refit=True)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train_transformed, cluster_labels)

print(neigh.predict(X_test_transformed))
print(neigh.predict_proba(X_test_transformed))

[59 59 58 74 90 74 58 58 59 58 90 59  0 58 58 59 58 59 30 74 58 74 59]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
#TODO: remap with X_train/test and the CLUSTER LABELS, not this

from sklearn.neighbors import NearestNeighbors

# Find nearest neighbors and their distances
n_neighbors = 5 
nn_model = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto')
nn_model.fit(X_train_transformed, y_train)
distances, indices = nn_model.kneighbors(X_test_transformed)

In [ ]:
# Add nearest neighbor indices to test dataframe
output_df = test_df.copy()
nn_df = pd.DataFrame(indices)
output_df['indices'] = nn_df[nn_df.columns[0:]].apply(lambda x: ';'.join(x.dropna().astype(str)), axis=1)
output_df["KNN_label"] = neigh.predict(X_test_transformed)

# Explode the indices
output_df["indices"] = output_df["indices"].str.split(';')
output_df = output_df.explode("indices")

# Add indices as column for the training dataframe
train_index = train_df.copy()
train_index["indices"] = train_index.index
train_index["indices"] = train_index["indices"].astype(str)

# Merge our dataframes to provide labels
left = output_df.copy()
right = train_index[["indices", "Target", "Cluster", "taxon_id"]]
right = right.rename(columns={"Target": "neighbor_media_id", "taxon_id": "neighbor_taxon_id", "Cluster": "neighbor_cluster"})
merged = pd.merge(left, right, on="indices", how="left")



#merged.to_csv(os.path.join(DATA_DIR, "ml-model-output.csv"), index=False)
merged

,Component 1,Component 2,Set,Target,Cluster,taxon_id,indices,KNN_label,neighbor_media_id,neighbor_cluster,neighbor_taxon_id
0,8.017917,10.637187,Test,unknown,59,KBase_derived_Bin.001.fastaBA_F_extracted_bins...,8080,59,J12,59,"[{'NCBI tax id': 562, 'Matching level': 'speci..."
1,8.017917,10.637187,Test,unknown,59,KBase_derived_Bin.001.fastaBA_F_extracted_bins...,2116,59,J168,59,1886799
2,8.017917,10.637187,Test,unknown,59,KBase_derived_Bin.001.fastaBA_F_extracted_bins...,7411,59,J19,59,"[{'NCBI tax id': 216816, 'Matching level': 'sp..."
3,8.017917,10.637187,Test,unknown,59,KBase_derived_Bin.001.fastaBA_F_extracted_bins...,2842,59,J168,59,228414
4,8.017917,10.637187,Test,unknown,59,KBase_derived_Bin.001.fastaBA_F_extracted_bins...,7410,59,J13,59,"[{'NCBI tax id': 216816, 'Matching level': 'sp..."
...,...,...,...,...,...,...,...,...,...,...,...
110,8.294023,10.321620,Test,unknown,59,KBase_derived_Bin.028.fastaBA_F_extracted_bins...,538,59,J280,59,1295125
111,8.294023,10.321620,Test,unknown,59,KBase_derived_Bin.028.fastaBA_F_extracted_bins...,3621,59,J280,59,311400
112,8.294023,10.321620,Test,unknown,59,KBase_derived_Bin.028.fastaBA_F_extracted_bins...,3914,59,J280,59,35749
113,8.294023,10.321620,Test,unknown,59,KBase_derived_Bin.028.fastaBA_F_extracted_bins...,7493,59,J118,59,"[{'NCBI tax id': 261965, 'Matching level': 'sp..."


### Random Forest

In [ ]:
y_train = cluster_labels
y_test = test_clusters

In [ ]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

model = RandomForestClassifier(
    n_estimators=100,
    n_jobs=-1,
    random_state=RDM
)

model.fit(X_train_transformed, y_train)

RandomForestClassifier(n_jobs=-1, random_state=47)

In [ ]:
report = classification_report(
    y_true=y_test,
    y_pred=model.predict(X_test_transformed),
    zero_division="warn"
)

print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
          30       1.00      1.00      1.00         1
          58       1.00      0.88      0.93         8
          59       1.00      1.00      1.00         7
          74       1.00      1.00      1.00         4
          90       0.67      1.00      0.80         2

    accuracy                           0.96        23
   macro avg       0.94      0.98      0.96        23
weighted avg       0.97      0.96      0.96        23



In [ ]:
from sklearn import metrics

roc_score = metrics.roc_auc_score(
    y_true=y_test,
    y_score=model.predict_proba(X_test_transformed),
    labels=np.unique(y_train),
    average="weighted",
    multi_class="ovo"
)

print("ROC AUC score:", roc_score)

ROC AUC score: 1.0
